In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor
from ml_models import evaluate_model,kNN, use_kNN, use_bagging
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn import svm


In [2]:
#preberi keggle podatke in shrani tabelico v dataframe spremenljivko
df = pd.read_csv("data/train.csv")
df

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0000,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0000,8,32,40
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0000,5,27,32
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0000,3,10,13
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0000,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
10881,2012-12-19 19:00:00,4,0,1,1,15.58,19.695,50,26.0027,7,329,336
10882,2012-12-19 20:00:00,4,0,1,1,14.76,17.425,57,15.0013,10,231,241
10883,2012-12-19 21:00:00,4,0,1,1,13.94,15.910,61,15.0013,4,164,168
10884,2012-12-19 22:00:00,4,0,1,1,13.94,17.425,61,6.0032,12,117,129


In [25]:
df.dtypes

datetime      datetime64[ns]
season              category
holiday             category
workingday          category
weather             category
temp                 float64
atemp                float64
humidity               int64
windspeed            float64
dayofweek           category
month               category
hour                   int64
dtype: object

In [3]:
#casual+registered=count, so delete them
df = df.drop(columns=["casual", "registered"])
#spremeni stolpec datetime ki je object v tip datetime
df["datetime"] = pd.to_datetime(df["datetime"])
#dodaj stolpec day of week ki nam pove kateri dan
df["dayofweek"] = df["datetime"].dt.dayofweek
#kateri mesec
df["month"] = df["datetime"].dt.month
#katera ura 
df["hour"] = df["datetime"].dt.hour
#spremeni vse kategoricne v kategoricne
df["season"] = df["season"].astype("category")
df["holiday"] = df["holiday"].astype("category")
df["workingday"] = df["workingday"].astype("category")
df["weather"] = df["weather"].astype("category")
df["dayofweek"] = df["dayofweek"].astype("category")
df["month"] = df["month"].astype("category")
df




,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,count,dayofweek,month,hour
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0000,16,5,1,0
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0000,40,5,1,1
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0000,32,5,1,2
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0000,13,5,1,3
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0000,1,5,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10881,2012-12-19 19:00:00,4,0,1,1,15.58,19.695,50,26.0027,336,2,12,19
10882,2012-12-19 20:00:00,4,0,1,1,14.76,17.425,57,15.0013,241,2,12,20
10883,2012-12-19 21:00:00,4,0,1,1,13.94,15.910,61,15.0013,168,2,12,21
10884,2012-12-19 22:00:00,4,0,1,1,13.94,17.425,61,6.0032,129,2,12,22


In [4]:
#pop outaj count stolpec v y
y = df.pop("count")
y
df

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,dayofweek,month,hour
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0000,5,1,0
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0000,5,1,1
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0000,5,1,2
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0000,5,1,3
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0000,5,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...
10881,2012-12-19 19:00:00,4,0,1,1,15.58,19.695,50,26.0027,2,12,19
10882,2012-12-19 20:00:00,4,0,1,1,14.76,17.425,57,15.0013,2,12,20
10883,2012-12-19 21:00:00,4,0,1,1,13.94,15.910,61,15.0013,2,12,21
10884,2012-12-19 22:00:00,4,0,1,1,13.94,17.425,61,6.0032,2,12,22


In [5]:
#hour kot numericna zvezna ko sta 00 in 24 najbolj oddaljena
df_time = df.copy()
#izbrisi stolpec datetime
df_time = df_time.drop(columns=["datetime"])
df_time





,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,dayofweek,month,hour
0,1,0,0,1,9.84,14.395,81,0.0000,5,1,0
1,1,0,0,1,9.02,13.635,80,0.0000,5,1,1
2,1,0,0,1,9.02,13.635,80,0.0000,5,1,2
3,1,0,0,1,9.84,14.395,75,0.0000,5,1,3
4,1,0,0,1,9.84,14.395,75,0.0000,5,1,4
...,...,...,...,...,...,...,...,...,...,...,...
10881,4,0,1,1,15.58,19.695,50,26.0027,2,12,19
10882,4,0,1,1,14.76,17.425,57,15.0013,2,12,20
10883,4,0,1,1,13.94,15.910,61,15.0013,2,12,21
10884,4,0,1,1,13.94,17.425,61,6.0032,2,12,22


In [6]:
#dodaj hour kot numericno zvezno ciklicno in ustvari novo tabelo df_ctime
df_ctime = df.copy()
#spremeni v polarne koordinate
df_ctime["hour"] = (df_ctime["hour"]*2*np.pi)/24
df_ctime["hour_sin"] = np.sin(df_ctime["hour"])
df_ctime["hour_cos"] = np.cos(df_ctime["hour"])
#izbrisi stolpca datetime in hour
df_ctime = df_ctime.drop(columns=["hour", "datetime"])
df_ctime

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,dayofweek,month,hour_sin,hour_cos
0,1,0,0,1,9.84,14.395,81,0.0000,5,1,0.000000,1.000000
1,1,0,0,1,9.02,13.635,80,0.0000,5,1,0.258819,0.965926
2,1,0,0,1,9.02,13.635,80,0.0000,5,1,0.500000,0.866025
3,1,0,0,1,9.84,14.395,75,0.0000,5,1,0.707107,0.707107
4,1,0,0,1,9.84,14.395,75,0.0000,5,1,0.866025,0.500000
...,...,...,...,...,...,...,...,...,...,...,...,...
10881,4,0,1,1,15.58,19.695,50,26.0027,2,12,-0.965926,0.258819
10882,4,0,1,1,14.76,17.425,57,15.0013,2,12,-0.866025,0.500000
10883,4,0,1,1,13.94,15.910,61,15.0013,2,12,-0.707107,0.707107
10884,4,0,1,1,13.94,17.425,61,6.0032,2,12,-0.500000,0.866025


In [7]:
#dodaj hour kot numericno diskretno cez 24 kategorij za vsako uro ena kategorija in ustvari novo tabelo df_dtime24
df_dtime24 = df.copy()
df_dtime24["hour"] = df["hour"].astype("category")
#deletaj datetime
df_dtime24 = df_dtime24.drop(columns=["datetime"])
df_dtime24


,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,dayofweek,month,hour
0,1,0,0,1,9.84,14.395,81,0.0000,5,1,0
1,1,0,0,1,9.02,13.635,80,0.0000,5,1,1
2,1,0,0,1,9.02,13.635,80,0.0000,5,1,2
3,1,0,0,1,9.84,14.395,75,0.0000,5,1,3
4,1,0,0,1,9.84,14.395,75,0.0000,5,1,4
...,...,...,...,...,...,...,...,...,...,...,...
10881,4,0,1,1,15.58,19.695,50,26.0027,2,12,19
10882,4,0,1,1,14.76,17.425,57,15.0013,2,12,20
10883,4,0,1,1,13.94,15.910,61,15.0013,2,12,21
10884,4,0,1,1,13.94,17.425,61,6.0032,2,12,22


In [8]:
#dodaj hour kot numbericno diskretno spremenljivko z manj kategorijami, ki ustrezajo grafu porazdelitve
#kategorije: 1,2,3,4,5,6 - 1
#7,8 - 2
#9,10 - 3
#11,12,13,14,15,16 - 4
#17,18 - 5
#19,20 - 6
#21,22,23,0 - 7
df_dtime_categ = df.copy()
prvi = [1,2,3,4,5,6]
drugi = [7,8]
tretji = [9,10]
cetrti = [11,12,13,14,15,16]
peti = [17,18]
sesti = [19,20]
sedmi = [21,22,23,0]
hours = df_dtime_categ["hour"]
updated_hours = []
for row in hours:
    if row in prvi:
        updated_hours.append(1);
    elif row in drugi:
        updated_hours.append(2);
    elif row in tretji:
        updated_hours.append(3);
    elif row in cetrti:
        updated_hours.append(4);
    elif row in peti: 
        updated_hours.append(5);
    elif row in sesti:
        updated_hours.append(6);
    elif row in sedmi:
        updated_hours.append(7);

df_dtime_categ["hour"] = updated_hours       
#kategoriziraj ure in deletaj datetime
df_dtime_categ = df_dtime_categ.drop(columns=["datetime"])
df_dtime_categ["hour"] = df_dtime_categ["hour"].astype("category")
df_dtime_categ

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,dayofweek,month,hour
0,1,0,0,1,9.84,14.395,81,0.0000,5,1,7
1,1,0,0,1,9.02,13.635,80,0.0000,5,1,1
2,1,0,0,1,9.02,13.635,80,0.0000,5,1,1
3,1,0,0,1,9.84,14.395,75,0.0000,5,1,1
4,1,0,0,1,9.84,14.395,75,0.0000,5,1,1
...,...,...,...,...,...,...,...,...,...,...,...
10881,4,0,1,1,15.58,19.695,50,26.0027,2,12,6
10882,4,0,1,1,14.76,17.425,57,15.0013,2,12,6
10883,4,0,1,1,13.94,15.910,61,15.0013,2,12,7
10884,4,0,1,1,13.94,17.425,61,6.0032,2,12,7


In [9]:
#razbij kategorizorane stolpce
df_time = pd.get_dummies(df_time, drop_first=True)
df_ctime = pd.get_dummies(df_ctime, drop_first=True)

df_dtime24 = pd.get_dummies(df_dtime24, drop_first=True)
df_dtime_categ = pd.get_dummies(df_dtime_categ, drop_first=True)
df_time.shape

(10886, 30)

In [10]:
#pretvori dataframe v numpy
df_time_np = df_time.values
df_ctime_np = df_ctime.values

df_dtime24_np = df_dtime24.values
df_dtime_categ_np = df_dtime_categ.values

y = y.values

In [11]:
#standardiziraj podatke
scaler = StandardScaler()
df_time_s = scaler.fit_transform(df_time_np) 
df_ctime_s = scaler.fit_transform(df_ctime_np) 

df_dtime24_s = scaler.fit_transform(df_dtime24_np) 
df_dtime_categ_s = scaler.fit_transform(df_dtime_categ_np) 

In [12]:
n_splits = 5

In [13]:
df_time_results = []
#linear regression ridge
df_time_results.append(evaluate_model(n_splits, linear_model.Ridge(alpha=1), df_time_s,y))
#decision tree
df_time_results.append(evaluate_model(n_splits, DecisionTreeRegressor(random_state=0), df_time_s,y))
#random forrest
df_time_results.append(evaluate_model(n_splits, RandomForestRegressor(random_state=0, n_estimators = 100), df_time_s,y))
#gradient boosting
df_time_results.append(evaluate_model(n_splits, GradientBoostingRegressor(random_state=0), df_time_s,y))
#neural network
df_time_results.append(evaluate_model(n_splits, MLPRegressor(random_state=0, max_iter=5000), df_time_s,y))
#svr
df_time_results.append(evaluate_model(n_splits, svm.SVR(), df_time_s,y))



In [14]:
df_ctime_results = []
#linear regression ridge
df_ctime_results.append(evaluate_model(n_splits, linear_model.Ridge(alpha=1), df_ctime_s,y))
#decision tree
df_ctime_results.append(evaluate_model(n_splits, DecisionTreeRegressor(random_state=0), df_ctime_s,y))
#random forrest
df_ctime_results.append(evaluate_model(n_splits, RandomForestRegressor(random_state=0, n_estimators = 100), df_ctime_s,y))
#gradient boosting
df_ctime_results.append(evaluate_model(n_splits, GradientBoostingRegressor(random_state=0), df_ctime_s,y))
#neural network
df_ctime_results.append(evaluate_model(n_splits, MLPRegressor(random_state=0, max_iter=3000), df_ctime_s,y))
#svr
df_ctime_results.append(evaluate_model(n_splits, svm.SVR(), df_ctime_s,y))

In [15]:
df_dtime24_results = []
#linear regression ridge
df_dtime24_results.append(evaluate_model(n_splits, linear_model.Ridge(alpha=1), df_dtime24_s,y))
#decision tree
df_dtime24_results.append(evaluate_model(n_splits, DecisionTreeRegressor(random_state=0), df_dtime24_s,y))
#random forrest
df_dtime24_results.append(evaluate_model(n_splits, RandomForestRegressor(random_state=0, n_estimators = 100), df_dtime24_s,y))
#gradient boosting
df_dtime24_results.append(evaluate_model(n_splits, GradientBoostingRegressor(random_state=0), df_dtime24_s,y))
#neural network
df_dtime24_results.append(evaluate_model(n_splits, MLPRegressor(random_state=0, max_iter=2000), df_dtime24_s,y))
#svr
df_dtime24_results.append(evaluate_model(n_splits, svm.SVR(), df_dtime24_s,y))

In [16]:
df_dtime_categ_results = []
#linear regression ridge
df_dtime_categ_results.append(evaluate_model(n_splits, linear_model.Ridge(alpha=1), df_dtime_categ_s,y))
#decision tree
df_dtime_categ_results.append(evaluate_model(n_splits, DecisionTreeRegressor(random_state=0), df_dtime_categ_s,y))
#random forrest
df_dtime_categ_results.append(evaluate_model(n_splits, RandomForestRegressor(random_state=0, n_estimators = 100), df_dtime_categ_s,y))
#gradient boosting
df_dtime_categ_results.append(evaluate_model(n_splits, GradientBoostingRegressor(random_state=0), df_dtime_categ_s,y))
#neural network
df_dtime_categ_results.append(evaluate_model(n_splits, MLPRegressor(random_state=0, max_iter=2000), df_dtime_categ_s,y))
#svr
df_dtime_categ_results.append(evaluate_model(n_splits, svm.SVR(), df_dtime_categ_s,y))

In [18]:
#kNN
n_splits = 5
K = 15

df_time_results.append(use_kNN(n_splits, K, df_time_s, y))
df_ctime_results.append(use_kNN(n_splits, K, df_ctime_s, y))
df_dtime24_results.append(use_kNN(n_splits, K, df_dtime24_s, y))
df_dtime_categ_results.append(use_kNN(n_splits, K, df_dtime_categ_s, y))

In [20]:
#bagging
nr_of_bags = 100
percent = 60


#linear regression ridge bagging
model = linear_model.Ridge(alpha=1)
df_time_results.append(use_bagging(n_splits, df_time_s, y, model, nr_of_bags, percent))
df_ctime_results.append(use_bagging(n_splits, df_ctime_s, y, model, nr_of_bags, percent))
df_dtime24_results.append(use_bagging(n_splits, df_dtime24_s, y, model, nr_of_bags, percent))
df_dtime_categ_results.append(use_bagging(n_splits, df_dtime_categ_s, y, model, nr_of_bags, percent))

#kNN bagging
model = KNeighborsRegressor(n_neighbors=15, metric='euclidean')
df_time_results.append(use_bagging(n_splits, df_time_s, y, model, nr_of_bags, percent))
df_ctime_results.append(use_bagging(n_splits, df_ctime_s, y, model, nr_of_bags, percent))
df_dtime24_results.append(use_bagging(n_splits, df_dtime24_s, y, model, nr_of_bags, percent))
df_dtime_categ_results.append(use_bagging(n_splits, df_dtime_categ_s, y, model, nr_of_bags, percent))

#decision trees bagging
model = DecisionTreeRegressor(random_state=0)
df_time_results.append(use_bagging(n_splits, df_time_s, y, model, nr_of_bags, percent))
df_ctime_results.append(use_bagging(n_splits, df_ctime_s, y, model, nr_of_bags, percent))
df_dtime24_results.append(use_bagging(n_splits, df_dtime24_s, y, model, nr_of_bags, percent))
df_dtime_categ_results.append(use_bagging(n_splits, df_dtime_categ_s, y, model, nr_of_bags, percent))


In [70]:
print(df_time_results)
print(df_ctime_results)
print(df_dtime24_results)
print(df_dtime_categ_results)

[(145.52549157871232, 0.8034666269897803), (90.11759090472142, 0.49755184477420555), (66.2593409119145, 0.36582710404566343), (82.33897720417283, 0.4546050287570556), (389.7373773727798, 2.151794662313013), (161.80480202854537, 0.8933469806993032), (139.95160506508884, 0.7726924186518372), (145.5128322041126, 0.803396732809049), (143.68526870754349, 0.7933064987045737), (65.96077865962404, 0.3641787000224786)]
[(134.09675090651302, 0.7403669485828108), (90.82699691559628, 0.5014685747473567), (66.74742358365661, 0.3685218768562722), (82.67753296744536, 0.4564742425574253), (98.70404645543101, 0.5449588688239735), (152.7603365711173, 0.8434112197877879), (120.23012517388781, 0.6638073652118393), (134.09426389363927, 0.7403532174363068), (128.87996832163202, 0.7115643610653777), (66.9351828640417, 0.36955852214817586)]
[(109.48134741415473, 0.6044618573067565), (105.21498313859895, 0.5809066624278139), (76.39049771421078, 0.42176264011662545), (91.71079269212552, 0.5063481350485269), (70

In [71]:
df_time_results_1, df_time_results_2 = zip(*df_time_results)
df_ctime_results_1, df_ctime_results_2 = zip(*df_ctime_results)
df_dtime24_results_1, df_dtime24_results_2 = zip(*df_dtime24_results)
df_dtime_categ_results_1, df_dtime_categ_results_2 = zip(*df_dtime_categ_results)

In [72]:
#write results in latex

In [73]:
results_rmse = np.array([df_time_results_1, df_ctime_results_1, df_dtime24_results_1, df_dtime_categ_results_1] )
df_results_rmse= pd.DataFrame(results_rmse,
                   columns=['Ridge Regression', 'Decision Trees', 'Random Forrest', 'Gradient Boosting', 'Neural Network', 'Support Vector Regression', 'K-Nearest Neighbour', 'Bagging with Ridge Regression', 'Bagging with K-Nearest Neighbour', 'Bagging with Decision Trees'], index = ['df_time', 'df_ctime', 'df_dtime24', 'df_dtime_categ'])
df_results_rmse = df_results_rmse.T

In [74]:
columns = df_results_rmse.columns
s = "& " + " & ".join(columns) + " \\\\\n"
for row_name, row in df_results_rmse.iterrows():
    s += row_name
    for c in columns:
        s += " & " + str(round(row[c], 2))
    s += " \\\\\n"
print(s)

& df_time & df_ctime & df_dtime24 & df_dtime_categ \\
Ridge Regression & 145.53 & 134.1 & 109.48 & 115.03 \\
Decision Trees & 90.12 & 90.83 & 105.21 & 106.1 \\
Random Forrest & 66.26 & 66.75 & 76.39 & 78.98 \\
Gradient Boosting & 82.34 & 82.68 & 91.71 & 89.27 \\
Neural Network & 389.74 & 98.7 & 70.14 & 83.23 \\
Support Vector Regression & 161.8 & 152.76 & 149.95 & 149.95 \\
K-Nearest Neighbour & 139.95 & 120.23 & 93.42 & 99.28 \\
Bagging with Ridge Regression & 145.51 & 134.09 & 109.47 & 115.05 \\
Bagging with K-Nearest Neighbour & 143.69 & 128.88 & 98.82 & 111.07 \\
Bagging with Decision Trees & 65.96 & 66.94 & 76.38 & 78.32 \\



In [75]:
results_ratio = np.array([df_time_results_2, df_ctime_results_2, df_dtime24_results_2, df_dtime_categ_results_2] )
df_results_ratio = pd.DataFrame(results_ratio,
                   columns=['Ridge Regression', 'Decision Trees', 'Random Forrest', 'Gradient Boosting', 'Neural Network', 'Support Vector Regression', 'K-Nearest Neighbour', 'Bagging with Ridge Regression', 'Bagging with K-Nearest Neighbour', 'Bagging with Decision Trees'], index = ['df_time', 'df_ctime', 'df_dtime24', 'df_dtime_categ'])
df_results_ratio = df_results_ratio.T
df_results_ratio

,df_time,df_ctime,df_dtime24,df_dtime_categ
Ridge Regression,0.803467,0.740367,0.604462,0.635111
Decision Trees,0.497552,0.501469,0.580907,0.585809
Random Forrest,0.365827,0.368522,0.421763,0.436068
Gradient Boosting,0.454605,0.456474,0.506348,0.492892
Neural Network,2.151795,0.544959,0.387260,0.459509
Support Vector Regression,0.893347,0.843411,0.827874,0.827892
K-Nearest Neighbour,0.772692,0.663807,0.515782,0.548135
Bagging with Ridge Regression,0.803397,0.740353,0.604417,0.635232
Bagging with K-Nearest Neighbour,0.793306,0.711564,0.545603,0.613228
Bagging with Decision Trees,0.364179,0.369559,0.421715,0.432409


In [76]:
columns = df_results_ratio.columns
s = "& " + " & ".join(columns) + " \\\\\n"
for row_name, row in df_results_ratio.iterrows():
    s += row_name
    for c in columns:
        s += " & " + str(round(row[c], 2))
    s += " \\\\\n"
print(s)

& df_time & df_ctime & df_dtime24 & df_dtime_categ \\
Ridge Regression & 0.8 & 0.74 & 0.6 & 0.64 \\
Decision Trees & 0.5 & 0.5 & 0.58 & 0.59 \\
Random Forrest & 0.37 & 0.37 & 0.42 & 0.44 \\
Gradient Boosting & 0.45 & 0.46 & 0.51 & 0.49 \\
Neural Network & 2.15 & 0.54 & 0.39 & 0.46 \\
Support Vector Regression & 0.89 & 0.84 & 0.83 & 0.83 \\
K-Nearest Neighbour & 0.77 & 0.66 & 0.52 & 0.55 \\
Bagging with Ridge Regression & 0.8 & 0.74 & 0.6 & 0.64 \\
Bagging with K-Nearest Neighbour & 0.79 & 0.71 & 0.55 & 0.61 \\
Bagging with Decision Trees & 0.36 & 0.37 & 0.42 & 0.43 \\

